# ECG Signal Detection with YOLO

We manually annotated 100 ECG images to train a YOLOv8 model for detecting 4 signal strips per image. The trained model weights are used for inference.

In [ ]:
!pip install ultralytics --quiet
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import random

# Load trained model
model = YOLO('/kaggle/input/signal-detector/ecg_signal_detector2/weights/best.pt')

# Select 4 random test images
test_images = list(Path('/kaggle/input/physionet-ecg-image-digitization/train').rglob('*.png'))[:1000]  # Sample from first folders
random.seed(13)
selected_images = random.sample(test_images, 4)

print(f"Model loaded. Testing on {len(selected_images)} images.")

## Raw YOLO Predictions (No Filtering)

Using default confidence threshold (0.25), the model sometimes:
- Detects <4 signals (misses some)
- Detects >4 signals (false positives)
- Detects partial signals instead of full strips

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, img_path in enumerate(selected_images):
    # Load image
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Run YOLO with default settings
    results = model(str(img_path), conf=0.25, verbose=False)
    boxes = results[0].boxes
    
    # Draw boxes
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()
        cv2.rectangle(img_rgb, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 3)
        cv2.putText(img_rgb, f'{conf:.2f}', (int(x1), int(y1)-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
    
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'{Path(img_path).name}\n{len(boxes)} detections (expected 4)', fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

# Note:
Here it may look like all signals are detected properly by the YOLO model, but since i found to be defetcs, i have added below post-training steps to the model.

## Smart Filtering Strategy

Based on annotation statistics:
- All images have exactly 4 signals
- Signal boxes cover ~86% of image width (σ=5.9%)
- Box widths within same image vary <0.5%

**Filtering approach:**
1. Lower confidence threshold to 0.15 (catch all signals)
2. Filter boxes by width (≥65% of image width)
3. Keep top 4 boxes by confidence
4. Sort by Y-coordinate (top→bottom = Lead 1→4)

In [ ]:
def smart_filter_boxes(boxes, img_width, img_height, min_width_ratio=0.65, target_count=4):
    """Filter and sort boxes to get exactly 4 signal strips."""
    if len(boxes) == 0:
        return []
    
    # Extract box data
    box_data = []
    for box in boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = box.conf[0].cpu().numpy()
        width = x2 - x1
        y_center = (y1 + y2) / 2
        
        box_data.append({
            'x1': int(x1), 'y1': int(y1), 'x2': int(x2), 'y2': int(y2),
            'conf': float(conf), 'y_center': y_center,
            'width_ratio': width / img_width
        })
    
    # Filter by width
    filtered = [b for b in box_data if b['width_ratio'] >= min_width_ratio]
    
    # Keep top N by confidence
    if len(filtered) > target_count:
        filtered = sorted(filtered, key=lambda b: b['conf'], reverse=True)[:target_count]
    
    # Sort by Y position (top to bottom)
    filtered = sorted(filtered, key=lambda b: b['y_center'])
    
    return filtered

## Filtered Predictions

Applying smart filtering to ensure exactly 4 signals are detected.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]  # Different color per lead

for idx, img_path in enumerate(selected_images):
    # Load image
    img = cv2.imread(str(img_path))
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_h, img_w = img.shape[:2]
    
    # Run YOLO with lower threshold
    results = model(str(img_path), conf=0.15, iou=0.5, verbose=False)
    boxes = results[0].boxes
    
    # Apply smart filtering
    filtered_boxes = smart_filter_boxes(boxes, img_w, img_h)
    
    # Draw filtered boxes
    for i, box in enumerate(filtered_boxes):
        x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
        conf = box['conf']
        color = colors[i % 4]
        
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color, 3)
        cv2.putText(img_rgb, f'Lead {i+1}: {conf:.2f}', (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
    title_color = 'green' if len(filtered_boxes) == 4 else 'red'
    axes[idx].imshow(img_rgb)
    axes[idx].set_title(f'{Path(img_path).name}\n{len(filtered_boxes)}/4 signals detected',
                       fontsize=10, color=title_color)
    axes[idx].axis('off')

plt.tight_layout()
plt.show()

## Results

Smart filtering achieves 100% success rate on validation set (20 images):
- **mAP50**: 0.937
- **mAP50-95**: 0.589
- **Precision**: 0.886
- **Recall**: 0.871

The filtered approach ensures exactly 4 signals are detected and correctly ordered (top→bottom).

In [ ]:
# Example: Extract signals from a single image
img_path = selected_images[0]
img = cv2.imread(str(img_path))
img_h, img_w = img.shape[:2]

results = model(str(img_path), conf=0.15, verbose=False)
boxes = results[0].boxes
filtered_boxes = smart_filter_boxes(boxes, img_w, img_h)

# Crop and display each signal
fig, axes = plt.subplots(1, 4, figsize=(16, 3))
for i, box in enumerate(filtered_boxes):
    x1, y1, x2, y2 = box['x1'], box['y1'], box['x2'], box['y2']
    signal_crop = img[y1:y2, x1:x2]
    signal_rgb = cv2.cvtColor(signal_crop, cv2.COLOR_BGR2RGB)
    
    axes[i].imshow(signal_rgb)
    axes[i].set_title(f'Signal {i+1}', fontsize=10)
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"Extracted {len(filtered_boxes)} signals from {Path(img_path).name}")